In [51]:
import numpy as np
import numpy.random as npr
import cvxpy as cp
import time

### Define Functions

In [3]:
def greedy_lower_bound_1(K, w, r): # Greedy Lower bound
    m,n = K.shape
    K_bar = 1-K
    v = np.zeros(n)
    for i in range(n):
        v[i] = np.ones(m) @ K_bar[:, i] * w[i]

    v_srtd = np.flip(np.argsort(v))
    v[v_srtd]

    # v_srtd is the arguments of the "most important" features to pick

    x = np.zeros(n)
    for i in range(n):
        x_temp = np.copy(x)
        x_temp[v_srtd[i]] = 1
        if np.count_nonzero(K @ x_temp) <= m*(1-r):
            x = x_temp

    # COUNT IS LOWER BOUND
    count = x @ w
    return count, x

In [4]:
def greedy_lower_bound_2(K, w, r): # Greedy Lower bound
    m,n = K.shape
    K_bar = 1-K
    v = np.zeros(n)
    for i in range(n):
        v[i] = np.ones(m) @ (1 / (K[:, i] + 1)) * w[i]

    v_srtd = np.flip(np.argsort(v))
    v[v_srtd]

    # v_srtd is the arguments of the "most important" features to pick

    x = np.zeros(n)
    for i in range(n):
        x_temp = np.copy(x)
        x_temp[v_srtd[i]] = 1
        if np.count_nonzero(K @ x_temp) <= m*(1-r):
            x = x_temp

    # COUNT IS LOWER BOUND
    count = x @ w
    return count, x

In [5]:
#DYNAMIC PROGRAMMING ALGORITHM

In [6]:
def solve_mip_upper_bound(K, w, r): # LP upper bound
    m,n = K.shape
    K_bar = 1 - K
    x = cp.Variable(n, boolean = True)
    obj = cp.Maximize(w @ x)
    # cons = [cp.norm(K @ x, p = 0) <= m * (1-r)]
    cons = [r * m * cp.sum(x) - np.ones(m) @ K_bar @ x <= 0]
    # cons += [x <= 1, 0 <= x]

    prob = cp.Problem(obj, cons)
    prob.solve()
    return prob.value, x.value

In [7]:
def solve_mip_lower_bound(K, w, r): # LP upper bound
    m,n = K.shape
    x = cp.Variable(n, boolean = True)
    obj = cp.Maximize(w @ x)
    # cons = [cp.norm(K @ x, p = 0) <= m * (1-r)]
    cons = [np.ones(m) @ K @ x <= m * (1-r)]
    # cons += [x <= 1, 0 <= x]

    prob = cp.Problem(obj, cons)
    prob.solve()
    return prob.value, x.value

In [8]:
def sdp_relaxation(K,w,r): # poor sdp relaxation
    m,n = K.shape
    K_bar = 1 - K
    X = cp.Variable((n+m+1,n+m+1), PSD = True)

    obj = cp.Maximize(w @ X[n+m,:n])
    cons = []
    cons += [cp.sum(X[n+m,n:n+m]) >= r*m]

    for i in range(n):
        cons += [X[i,i] == X[n+m,i]]
    for j in range(n, n+m):
        cons += [X[j,j] == X[n+m,j]]
    cons += [X[m+n,m+n] == 1]
    for i in range(n):
        for j in range(m):
            if K[j,i] == 1:
                cons += [X[i,n+j] == 0]
    # cons += [X[n+m,:n] <= 1, 0 <= X[n+m,:n]]
    cons += [X <= 1, 0 <= X]
    cons += [r * m * cp.sum(X[n+m,:n]) - np.ones(m) @ K_bar @ X[n+m,:n] <= 0]


    prob = cp.Problem(obj, cons)

    prob.solve()
    return prob.value, X.value

In [9]:
def true_const_sat(K, r, x): # is the actual constraint satisfied
    m,_ = K.shape
    return np.count_nonzero(K @ x) <= m*(1-r)

In [10]:
def expon_time_alg(K, w, r):
    m,n = K.shape
    x = np.zeros(n)
    best_val = 0
    for i in range(2 ** n):
        x_temp =np.array(list(np.binary_repr(i).zfill(n)), dtype=int) 
        if true_const_sat(K,r,x_temp) and w @ x_temp >= best_val:
            x = x_temp
            best_val = w @ x
    return best_val, x


In [55]:
def int_program(K, w, r):
    m,n = K.shape
    x = cp.Variable(n, boolean = True)
    y = cp.Variable(m)
    z = cp.Variable(m, boolean = True)
    
    u = np.sum(K, axis = 1)
    C = m * (1 - r)

    cons = [y >= 0, y <= cp.multiply(u,z)]
    cons += [cp.sum(z) <= C]
    cons += [K @ x == y]

    obj = cp.Maximize(w @ x)
    prob = cp.Problem(obj, cons)
    prob.solve()
    
    return prob.value, x.value

## Make random data matrix

In [70]:
npr.seed(2)

m = 300
n = 20
k = 0

D = npr.randint(2, size = (m,n))
K = np.abs(D - D[k,:])
K_bar = 1-K

# information weighting vector
w = npr.randint(0, 5, size = n)
r = 0.6

In [71]:

# p_val_low_g1, x_val_low_g1 = greedy_lower_bound_1(K,w,r)
# p_val_low_g2, x_val_low_g2 = greedy_lower_bound_2(K,w,r)
# p_val_mip_high, x_val_mip_high = solve_mip_upper_bound(K,w,r)
# p_val_mip_low, x_val_mip_low = solve_mip_lower_bound(K,w,r)

# bound_tight = true_const_sat(K, r, x_val_mip_high)

p_val_high_sdp, x_val_high_sdp = sdp_relaxation(K,w,r)

start_time = time.time()
true_opt_val, true_opt_sol = int_program(K, w, r)
end_time = time.time()

# true_opt_val, true_opt_sol = expon_time_alg(K, w, r)

In [67]:
# print("Upper bound MIP = ", p_val_mip_high)
# print("Lower bound MIP = ", p_val_mip_low)
print("Lower_bound G1 = ", p_val_low_g1)
print("Lower_bound G2 = ", p_val_low_g2)
print("upper bound sdp = ", p_val_high_sdp)
print("true value = ", true_opt_val)
# print("upper bound LP tight = ", true_const_sat(K,r, x_val_mip_high))

print("TOTAL TIME = ", end_time - start_time)



Lower_bound G1 =  7.0
Lower_bound G2 =  7.0
upper bound sdp =  25.500257744010607
true value =  8.0
TOTAL TIME =  0.011003255844116211


In [63]:
true_opt_sol @ w

8.0

In [64]:
x = cp.Variable(5)

constraints = [x[1]>=x[2],x[1]>=x[3],x[4]>=x[2],x[4]>=x[3]]
constraints += [x>=0, np.ones(5)@x ==1, x[0] == 0]
objective = cp.Minimize(7*x[1] + x[2] + 3*x[3] + 5*x[4])

prob = cp.Problem(objective, constraints)

In [22]:
prob.solve()
x.value

array([1.00083504e-13, 2.50000000e-01, 2.50000000e-01, 2.50000000e-01,
       2.50000000e-01])

In [147]:

mem = {}
def fib(n):
    global counter
    counter += 1
    if n in mem:
        return mem[n]
    elif n == 0 or n == 1:
        ans = 1
        mem[n] = 1
        return ans
    else:
        # ans = fib(n-1) + fib(n-2)
        a = fib(n-1)
        b = fib(n-2)
        ans = a + b
        mem[n] = ans
        return ans
    

counter = 0
fib(20)
print(counter)

39


89

19